In [1]:
import numpy as np
from dynamic_coding_analysis.analysis_pipeline import get_data
from tqdm import trange
from dynamic_coding_analysis.analysis_pipeline import process_spikes as spk

C:\Users\ilow1\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
base = 'G:/My Drive/Giocomo Lab/RandomForage/'
hist = np.load(base + 'aggregate_data/_histology.npy').item()
save_folder = base + 'aggregate_data/'

In [3]:
m = 'Lange'
s = '0308_speed_2'
data = {}
data[m] = {}
data[m][s] = {}
d = data[m][s]

In [4]:
folder = base + m + '/'
data_file = m + '_' + s + '_data.mat'
behavior = get_data.loadData(folder + data_file)
d['behavior'] = behavior

G:/My Drive/Giocomo Lab/RandomForage/Lange/Lange_0308_speed_2_data.mat
dict_keys(['sp', 'post', 'posx', 'lickt', 'trial', 'reward'])


In [5]:
# define good cells, store depths
# get spiking data
sp = behavior['sp']
cgs = sp['cgs'].copy()
cids = sp['cids'].copy()
cells = cids[cgs == 2]

# filter and sort by depth
if 'spike_depth' in sp.keys():
    depth = sp['spike_depth'].copy()
    depth_idx = np.argsort(sp['spike_depth'])
    cells = cells[depth_idx]
    sorted_depth = depth[depth_idx]
#     if 'histology' in hist[m][s].keys():
#         # get length of probe in MEC
#         coords = hist[m][s]['histology']
#         v = coords[0][:-1] - coords[1]
#         l = np.sqrt(np.sum(np.square(v)))
#         cells = cells[sorted_depth <= l]
#         d['depth'] = sorted_depth[sorted_depth <= l]
#     else:
    cells = cells[sorted_depth <= (np.max(depth) - 1000)]
    d['depth'] = sorted_depth[sorted_depth <= (np.max(depth) - 1000)]                
#     cells = cells[sorted_depth <= 2000]
#     d['depth'] = sorted_depth[sorted_depth <= 2000]                
    print(m + '_' + s + ': estimated MEC border as 1000um deep')
else:
    print(m + '_' + s + ': no depth info! All cells included and unsorted.')

d['cells'] = cells

Lange_0308_speed_2: estimated MEC border as 1000um deep


In [6]:
# get behavioral params
behave_data = d['behavior']
posx = behave_data['posx']
post = behave_data['post']
trial = get_data.trial_idx(posx)
dt = np.unique(np.round(np.diff(post),4))
speed = get_data.getSpeed(posx, dt)

# filter by trial (Toronto_1115 and Lange 0308)
if (m == 'Toronto') & (s == '1115_1'):
    trial_idx = trial < 300
    posx = posx[trial_idx]
    post = post[trial_idx]
    speed = speed[trial_idx]
    trial = trial[trial_idx]
elif (m == 'Lange') & (s == '0308_speed_2'):
    trial_idx = (trial <= 100) # & (trial >= 40)
    post = post[trial_idx]
#     post_offset = post[0]
#     post = post - post_offset
#     spiket = sp['st'].copy()
#     spiket = spiket - post_offset
#     sp['st'] = spiket
    posx = posx[trial_idx]    
    speed = speed[trial_idx]
    trial = trial[trial_idx]

# get neural params
cells = d['cells']
sp = behave_data['sp']
spiket = sp['st'].copy()
cluster_id = sp['clu'].copy()
if np.max(spiket) > np.max(post):
    cluster_id = cluster_id[spiket <= np.max(post)]
    spiket = spiket[spiket <= np.max(post)]

# format spike and behavioral data
A = np.column_stack((posx, speed, trial, post))
B = np.zeros((A.shape[0], cells.shape[0]))
print('spike matrix size pre behavioral filtering = ' + str(B.shape))

for i in trange(cells.shape[0]):
    # get spike times
    st = spiket[cluster_id == cells[i]]
    B[:, i] = get_data.spiketrain(post, dt, st)

# filter by speed
def find(x):
    return x.nonzero()[0]
speed_to_trash = find(speed < 2)
keep_idx = np.setdiff1d(np.arange(A.shape[0]), speed_to_trash)
A = A[keep_idx, :]
B = B[keep_idx, :]

# correct track ends
neg_pos = find(A[:, 0] < 0)
A[neg_pos, 0] = A[neg_pos, 0] + 400
plus_pos = find(A[:, 0] >= 400)
A[plus_pos, 0] = A[plus_pos, 0] - 400

# filter by spikes
cells = d['cells']
d['cells'] = cells[np.sum(B, axis=0) > 400]
B = B[:, np.sum(B, axis=0) > 400]

print('spike matrix size post behavioral filtering = ' + str(B.shape))
d['A'] = A
d['B'] = B

spike matrix size pre behavioral filtering = (100328, 56)


100%|███████████████████| 56/56 [00:00<00:00, 333.14it/s]


spike matrix size post behavioral filtering = (100328, 47)


In [7]:
np.sum(trial_idx)

100328

In [8]:
A = d['A']
B = d['B']
Y, centers = spk.tuning_curve_bytrial(A[:, 0], A[:, 2], B, dt, 5, smooth=True, normalize=False)
d['Y_raw'] = Y

In [9]:
Y = d['Y_raw']
Y[np.isnan(Y)] = 0.0

# normalize and clip at 90th percentile
for n in range(Y.shape[-1]):
    Yn = Y[:, :, n]
    Y[:, :, n] = np.clip(Yn, 0, np.percentile(Yn[Yn > 0], 90))
Y = Y - np.min(Y, axis=(0, 1), keepdims=True)
Y = Y / np.max(Y, axis=(0, 1), keepdims=True)
Y = Y[:, :, np.all(np.isfinite(Y), axis=(0, 1))]
d['Y'] = Y

In [10]:
# get file names
session_name = m + '_' + s
FR_file = session_name + '_MEC_FRtensor.npy'
spikes_file = session_name + '_MEC_spikes.npy'
behavior_file = session_name + '_behavior.npy'
cells_file = session_name + '_MEC_cellIDs.npy'

# save files
np.save(save_folder + 'gap_corrected/' + FR_file, d['Y'])
np.save(save_folder + 'gap_corrected/' + spikes_file, d['B'])
np.save(save_folder + 'gap_corrected/' + behavior_file, d['A'])    
np.save(save_folder + 'gap_corrected/' + cells_file, d['cells'])  